#### loading checkpoints

In [14]:
import gc
import hydra
import json
import sys
import os
import importlib
import random
import torch
import pickle
from collections import Counter, defaultdict
from typing import Optional
from pathlib import Path
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)
from peft import (
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModelForSeq2SeqLM,
)

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use("rose-pine")

In [4]:
# make proof_flow package available for imports
sys.path.append(os.path.abspath(os.path.join('..')))

from proof_flow.src.utils import (
    repo_root,
    prepare_environment_for_lean_dojo,
    get_config,
    set_up_padding,
)

prepare_environment_for_lean_dojo()
get_config.cache_clear() # clear cache to avoid stale config
main_cfg = get_config(config_name="train_five")
device = torch.device('cuda')

from lean_dojo import LeanGitRepo, Theorem, Dojo

2024-09-23 23:17:23,578	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
ckpt_dir = repo_root() / "checkpoints"

In [13]:
ckpt200dir = ckpt_dir / "200"
ckpt200_adapter_dir = ckpt200dir / "gfn_policy"


In [7]:
from transformers import AutoModel

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt200dir)

In [12]:
model

T5ForConditionalGeneration(
  (shared): Embedding(384, 1472)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1472)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1472, out_features=384, bias=False)
              (k): Linear(in_features=1472, out_features=384, bias=False)
              (v): Linear(in_features=1472, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=1472, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1472, out_features=3584, bias=False)
              (wi_1): Linear(in_features=1472, out_features=3584, bias=False)
              (w

In [20]:
peft_model = PeftModelForSeq2SeqLM.from_pretrained(
    model,
    ckpt200_adapter_dir,
    adapter_name="gfn_policy",
)

In [21]:
peft_model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(384, 1472)
      (encoder): T5Stack(
        (embed_tokens): Embedding(384, 1472)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1472, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                      (gfn_policy): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1472, out_features=32, bias=False)
                      (gfn_policy): Linear(in_features=1472, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                   

In [22]:
peft_model.active_adapter


'gfn_policy'

In [25]:
from huggingface_hub import login
from proof_flow.src.utils import get_hf_access_token

In [26]:
# login to the hub (needed for saving the model)
login(token=get_hf_access_token())

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/matthewho/.cache/huggingface/token
Login successful


In [27]:
peft_model.push_to_hub("msho/reprover_200step_ckpt")

adapter_model.safetensors:   0%|          | 0.00/50.1M [00:00<?, ?B/s]
adapter_model.safetensors:  12%|█▏        | 6.18M/50.1M [00:00<00:00, 61.8MB/s]

adapter_model.safetensors:  25%|██▍       | 12.4M/50.1M [00:00<00:01, 27.0MB/s]
adapter_model.safetensors:  54%|█████▍    | 27.2M/50.1M [00:01<00:00, 25.9MB/s]
adapter_model.safetensors:  62%|██████▏   | 30.9M/50.1M [00:01<00:00, 27.8MB/s]
adapter_model.safetensors:  68%|██████▊   | 34.3M/50.1M [00:01<00:00, 19.2MB/s]
adapter_model.safetensors:  93%|█████████▎| 46.7M/50.1M [00:01<00:00, 25.0MB/s]

adapter_model.safetensors:  99%|█████████▉| 49.6M/50.1M [00:02<00:00, 16.6MB/s]
adapter_model.safetensors: 100%|██████████| 50.1M/50.1M [00:02<00:00, 18.9MB/s]
adapter_model.safetensors: 100%|██████████| 50.1M/50.1M [00:02<00:00, 17.0MB/s]
Upload 2 LFS files: 100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


CommitInfo(commit_url='https://huggingface.co/msho/reprover_200step_ckpt/commit/db5599f69adfc4d187dc65250d126eda8335ae54', commit_message='Upload model', commit_description='', oid='db5599f69adfc4d187dc65250d126eda8335ae54', pr_url=None, repo_url=RepoUrl('https://huggingface.co/msho/reprover_200step_ckpt', endpoint='https://huggingface.co', repo_type='model', repo_id='msho/reprover_200step_ckpt'), pr_revision=None, pr_num=None)